In [127]:
import pandas as pd
import pickle
from pathlib import Path
import plotly.express as px
px.set_mapbox_access_token(open("mapbox_token.txt").read())

In [2]:
Path(Path.cwd(), "raw_data", "wind_power", "operational_list.pkl")

PosixPath('/Users/jamesmoro/Documents/Python/UK_Net_Zero/raw_data/wind_power/operational_list.pkl')

# Operational

In [3]:
operational_pickle = Path(Path.cwd(), "raw_data", "wind_power", "operational_list.pkl")
with open(operational_pickle, 'rb') as operational:
    operational_list = pickle.load(operational)

In [117]:
def return_substring(raw_string:str, substring:str) -> str:
    start_idx = raw_string.find(substring) + len(substring)
    return raw_string[start_idx:].strip()

def generate_project_info(raw_list:list) -> dict:
    project_info_dict = {}
    split_list = raw_list.split("\n")
    project_info_dict['Project Name'] = return_substring(split_list[0], "Name")
    project_info_dict['Location'] = return_substring(split_list[1], "Location")
    project_info_dict['County'] = return_substring(split_list[2] + " " + split_list[3], "Co.")
    project_info_dict['Turbine Capacity'] = float(return_substring(split_list[4], "Capacity"))
    project_info_dict['Number of Turbines'] = return_substring(split_list[5], "Turbines")
    project_info_dict['Project Capacity (MW)'] = float(return_substring(split_list[6], "(MW)"))
    project_info_dict['Developer'] = return_substring(split_list[7], "Developer")
    project_info_dict['Owner'] = return_substring(split_list[8], "Owner")
    project_info_dict['Local Planning Authority'] = return_substring(split_list[9], "Authority")
    
    end_idx = split_list[11].find("Longitude (decimal)")
    project_info_dict['Latitude'] = float(split_list[11][:end_idx].strip())
    
    project_info_dict['Longitude'] = float(return_substring(split_list[11], "decimal)"))
    
    temp_string = split_list[12] + " " + split_list[13]
    end_idx = temp_string.find("Project")
    project_info_dict['Type'] = temp_string[:end_idx].replace("Type", "").strip()
    
    project_info_dict['Project Status'] = return_substring(split_list[13], "Status")
    
    return project_info_dict

In [120]:
info_dict_list = []
errors = []
for i, _ in enumerate(operational_list):
    try:
        info_dict_list.append(generate_project_info(operational_list[i]))
    except (IndexError, ValueError) as e:
        errors.append(i)
        continue

In [121]:
df = pd.DataFrame(info_dict_list)

In [125]:
df.head(1)

,Project Name,Location,County,Turbine Capacity,Number of Turbines,Project Capacity (MW),Developer,Owner,Local Planning Authority,Latitude,Longitude,Type,Project Status
0,"Abbey Road, 209 Turbine 2","580m NE of 209 Abbey Road, Millisle",Down Region Northern Ireland,0.25,1,0.25,Wind NI Ltd,Albion Community Power plc,Ards and North Down Borough Council,54.5941,-5.53866,onshore,Operational


In [133]:
fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude",     
                        color="Project Capacity (MW)", 
                        hover_data = ["Project Name", "Number of Turbines", "Project Capacity (MW)"],
                  color_continuous_scale=px.colors.cyclical.IceFire, 
                  size_max=15, 
                  zoom=3)
fig.show()